In [1]:

import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, year # κτλ...

# 1. Start Spark
spark = SparkSession.builder \
    .appName("LA_Crime_Query_X") \
    .config("spark.executor.instances", "4") \
    .getOrCreate()

# 2. Define Path
DATA_PATH = "s3a://groups-bucket-dblab-905418150721/group36/processed_data"

# 3. Load Clean Data (Αυτό που φτιάξαμε στο Notebook 02)
df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_clean.parquet")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
739,application_1765289937462_0732,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# 1. Ξεκινάμε το Spark
spark = SparkSession.builder.appName("Query_Notebook").getOrCreate()

# 2. Ορίζουμε το Path (το δικό σου group path)
DATA_PATH = "s3a://groups-bucket-dblab-905418150721/group36/processed_data"

# 3. Φορτώνουμε τα ΚΑΘΑΡΑ δεδομένα (όχι τα raw)
# ΠΡΟΣΟΧΗ: Στο Notebook 1 το έλεγες 'df_crime_total'. 
# Εδώ θα το φορτώσουμε ως 'df_crime' (είναι πιο σύντομο).
print("Loading Clean Data...")
df_crime = spark.read.parquet(f"{DATA_PATH}/crime_data_clean.parquet")

# 4. Τυπώνουμε για επιβεβαίωση
print(f"Data Loaded. Rows: {df_crime.count()}")
df_crime.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Loading Clean Data...
Data Loaded. Rows: 3134980
root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true

In [3]:
QUERY 1 - Να ταξινομημηθούν, σε φθίνουσα σειρά, οι ηλικιακές ομάδες των θυμάτων σε περιστατικά που περι-
λαμβάνουν οποιαδήποτε μορφή “βαριάς σωματικής βλάβης”. Θεωρούμε 4 ηλικιακές ομάδες:
    1. Παιδιά <18 ετών
    2. Νεαροί ενήλικοι 18-24 ετών
    3.Ενήλικοι 25-64 ετών
    4.Ηλικιωμένοι >64 ετών

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    QUERY 1 - ?? ??????????????, ?? ???????? ?????, ?? ????????? ?????? ??? ??????? ?? ??????????? ??? ????-
          ^
SyntaxError: invalid syntax



In [4]:
from pyspark.sql.functions import upper

#Αναζήτηση για Aggravated Assault στη στήλη Crm Cd Desc
df_agg_assault = df_crime.filter(
    upper(col("Crm Cd Desc")).contains("AGGRAVATED ASSAULT")
)

count_agg_assault = df_agg_assault.count()
print(f"instances of Aggravated Assault: {count_agg_assault}")

#Αν βρέθηκαν αποτελέσματα, εμφάνιση παραδειγμάτων
if count_agg_assault > 0:
    print("\nInstances of Aggravated Assault:")
    df_agg_assault.select("Crm Cd Desc", "Vict Age").show(5, truncate=False)
else:
    print("\nno instances of Aggravated Assault found")
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

instances of Aggravated Assault: 177381

Instances of Aggravated Assault:
+----------------------------------------------+--------+
|Crm Cd Desc                                   |Vict Age|
+----------------------------------------------+--------+
|ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT|50      |
|ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT|56      |
|ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT|39      |
|ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT|58      |
|INTIMATE PARTNER - AGGRAVATED ASSAULT         |38      |
+----------------------------------------------+--------+
only showing top 5 rows

In [5]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

print(f"Original aggravated assault count: {df_agg_assault.count()}")

# First, let's check the data type of Vict Age
print("\nChecking Vict Age column type:")
df_agg_assault.printSchema()

# Show some raw values to see what we have
print("\nSample Vict Age values (raw):")
df_agg_assault.select("Vict Age").distinct().limit(10).show()

# Better approach: Handle nulls and invalid numbers
df_agg_clean = df_agg_assault.filter(
    col("Vict Age").isNotNull()
).filter(
    # Cast to string first for comparison, then back to int
    col("Vict Age").cast("string") != "0"
).filter(
    col("Vict Age").cast("string") != ""
)

print(f"After basic filtering: {df_agg_clean.count()}")

# Now convert to integer
df_agg_clean = df_agg_clean.withColumn(
    "Vict_Age_Int",
    col("Vict Age").cast(IntegerType())
)

# Filter for reasonable age range
df_agg_clean = df_agg_clean.filter(
    (col("Vict_Age_Int") > 0) & 
    (col("Vict_Age_Int") <= 120)
)

print(f"After age range filter (1-120): {df_agg_clean.count()}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Original aggravated assault count: 177381

Checking Vict Age column type:
root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3:

In [6]:

import time
from pyspark.sql.functions import col, when, count, desc

# 1. START TIMER
start_time = time.time()

print("Step 4: Grouping victims into age categories")

# (Assuming df_agg_clean exists from a previous step)

# Create age groups based on the specified ranges
df_age_groups = df_agg_clean.withColumn(
    "Age_Group",
    when(col("Vict_Age_Int") < 18, "Children (<18)")
    .when((col("Vict_Age_Int") >= 18) & (col("Vict_Age_Int") <= 24), "Young Adults (18-24)")
    .when((col("Vict_Age_Int") >= 25) & (col("Vict_Age_Int") <= 64), "Adults (25-64)")
    .when(col("Vict_Age_Int") > 64, "Elderly (>64)")
    .otherwise("Unknown")
)

print("Sample of age group assignment:")
df_age_groups.select("Vict_Age_Int", "Age_Group").show(10, truncate=False)

# Group by age group and count
df_age_counts = df_age_groups.groupBy("Age_Group") \
    .agg(count("*").alias("Victim_Count")) \
    .orderBy(desc("Victim_Count"))

# Cache this count so we don't recalculate it twice
total_victims = df_age_groups.count()

print(f"\nTotal victims analyzed: {total_victims}")
print("Victim count by age group (descending order):")
df_age_counts.show(truncate=False)
# Calculate percentage for each group
df_final_result = df_age_counts.withColumn(
    "Percentage",
    (col("Victim_Count") / total_victims * 100).cast("decimal(5,2)")
)

print("\nFinal result with percentages:")
df_final_result.show(truncate=False)

# Also show the distribution visually
print("\nDistribution summary:")
for row in df_final_result.collect():
    group = row["Age_Group"]
    # FIX: Renamed variable from 'count' to 'v_count' to avoid overwriting the PySpark function
    v_count = row["Victim_Count"] 
    perc = row["Percentage"]
    print(f"  {group}: {v_count} victims ({perc}%)")

print("\nStep 4 completed.")

# 2. END TIMER AND PRINT
print(f"--- Total Execution Time: {time.time() - start_time:.2f} seconds ---")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Step 4: Grouping victims into age categories
Sample of age group assignment:
+------------+--------------------+
|Vict_Age_Int|Age_Group           |
+------------+--------------------+
|50          |Adults (25-64)      |
|56          |Adults (25-64)      |
|39          |Adults (25-64)      |
|58          |Adults (25-64)      |
|38          |Adults (25-64)      |
|38          |Adults (25-64)      |
|46          |Adults (25-64)      |
|23          |Young Adults (18-24)|
|23          |Young Adults (18-24)|
|36          |Adults (25-64)      |
+------------+--------------------+
only showing top 10 rows


Total victims analyzed: 172271
Victim count by age group (descending order):
+--------------------+------------+
|Age_Group           |Victim_Count|
+--------------------+------------+
|Adults (25-64)      |121620      |
|Young Adults (18-24)|33741       |
|Children (<18)      |10899       |
|Elderly (>64)       |6011        |
+--------------------+------------+


Final result with percent

In [7]:
# Step 5: DataFrame API WITH UDF (FIXED)

from pyspark.sql.functions import udf, col, desc
from pyspark.sql.types import StringType
import time

print("Step 5: DataFrame API WITH UDF")
print("Starting execution time measurement...")

start_time = time.time()

# Define the UDF function
def categorize_age_udf(age):
    """User Defined Function to categorize age into groups"""
    try:
        age_int = int(age)
        if age_int < 18:
            return "Children (<18)"
        elif 18 <= age_int <= 24:
            return "Young Adults (18-24)"
        elif 25 <= age_int <= 64:
            return "Adults (25-64)"
        elif age_int > 64:
            return "Elderly (>64)"
        else:
            return "Unknown"
    except:
        return "Unknown"

# Register the UDF
age_group_udf = udf(categorize_age_udf, StringType())
# Apply UDF to create age groups
df_with_udf = df_agg_clean.withColumn(
    "Age_Group_UDF",
    age_group_udf(col("Vict_Age_Int"))
)

# Need to import count function again
from pyspark.sql.functions import count

# Group and count
df_udf_counts = df_with_udf.groupBy("Age_Group_UDF") \
    .agg(count("*").alias("Victim_Count")) \
    .orderBy(desc("Victim_Count"))

# Calculate percentages
total_udf = df_with_udf.count()
df_udf_final = df_udf_counts.withColumn(
    "Percentage",
    (col("Victim_Count") / total_udf * 100).cast("decimal(5,2)")
)

# Measure execution time
execution_time_udf = time.time() - start_time

print(f"\nUDF Results:")
df_udf_final.show(truncate=False)
print(f"Execution time with UDF: {execution_time_udf:.2f} seconds")

print("\nStep 5 completed (DataFrame API WITH UDF).")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Step 5: DataFrame API WITH UDF
Starting execution time measurement...

UDF Results:
+--------------------+------------+----------+
|Age_Group_UDF       |Victim_Count|Percentage|
+--------------------+------------+----------+
|Adults (25-64)      |121620      |70.60     |
|Young Adults (18-24)|33741       |19.59     |
|Children (<18)      |10899       |6.33      |
|Elderly (>64)       |6011        |3.49      |
+--------------------+------------+----------+

Execution time with UDF: 0.72 seconds

Step 5 completed (DataFrame API WITH UDF).

In [8]:
# Step 6: RDD API Implementation

from pyspark.sql import Row
import time

print("Step 6: RDD API Implementation")
print("Starting execution time measurement...")

rdd_start_time = time.time()

# Convert DataFrame to RDD
crime_rdd = df_agg_clean.rdd

print(f"RDD partition count: {crime_rdd.getNumPartitions()}")
print(f"RDD element count: {crime_rdd.count()}")

# Define mapping function
def map_to_age_group(row):
    """Map each row to (age_group, 1)"""
    age = row.Vict_Age_Int
    
    if age < 18:
        return ("Children (<18)", 1)
    elif 18 <= age <= 24:
        return ("Young Adults (18-24)", 1)
    elif 25 <= age <= 64:
        return ("Adults (25-64)", 1)
    elif age > 64:
        return ("Elderly (>64)", 1)
    else:
        return ("Unknown", 1)

# MapReduce operation
age_group_rdd = crime_rdd.map(map_to_age_group)

# Reduce by key (sum counts for each age group)
reduced_rdd = age_group_rdd.reduceByKey(lambda a, b: a + b)

# Sort by count in descending order
sorted_rdd = reduced_rdd.map(lambda x: (x[1], x[0])) \
                       .sortByKey(ascending=False) \
                       .map(lambda x: (x[1], x[0]))

# Collect results
rdd_results = sorted_rdd.collect()

# Calculate total for percentages
total_rdd = sum(count for _, count in rdd_results)

# Convert to list of Rows for DataFrame creation
result_rows = []
for age_group, count in rdd_results:
    percentage = (count / total_rdd * 100) if total_rdd > 0 else 0
    result_rows.append(Row(
        Age_Group=age_group,
        Victim_Count=count,
        Percentage=round(percentage, 2)
    ))

# Create DataFrame from RDD results
df_rdd_results = spark.createDataFrame(result_rows)

rdd_execution_time = time.time() - rdd_start_time
print(f"\nRDD API Results:")
df_rdd_results.show(truncate=False)
print(f"Execution time with RDD API: {rdd_execution_time:.2f} seconds")

print("\nStep 6 completed (RDD API).")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Step 6: RDD API Implementation
Starting execution time measurement...
RDD partition count: 8
RDD element count: 172271

RDD API Results:
+--------------------+------------+----------+
|Age_Group           |Victim_Count|Percentage|
+--------------------+------------+----------+
|Adults (25-64)      |121620      |70.6      |
|Young Adults (18-24)|33741       |19.59     |
|Children (<18)      |10899       |6.33      |
|Elderly (>64)       |6011        |3.49      |
+--------------------+------------+----------+

Execution time with RDD API: 11.87 seconds

Step 6 completed (RDD API).